In [0]:
#Creating a rude chatbot - thanks to Alexander Combs & Michael Roman for the base code
import pandas as pd
import re
pd.set_option('display.max_colwidth',200)

In [0]:
#Chatlogs from Cleverbot - a very rude chatbot
df = pd.read_csv('nscb.csv') #https://www.cleverbot.com/

In [3]:
#Segmenting out the chat data
convo = df.iloc[:,0]
convo

0        User: *walking*\nCleverbot: *looks at you and winks* ;)\nUser: O.O *walks faster*\nCleverbot: *catches up with you and pins you against a wall*\nUser: *eyes widen* W-What do you want?\nCleverbot: ...
1        User: *Facepalm* You are so stupid, you know that, right?\nCleverbot: I cannot argue with you.\nUser: So you admit your idiocy.\nCleverbot: You're all and I am nothing.\nUser: Yess!!!\nCleverbot: ...
2                                                                                      Cleverbot: Well, my wife left me today.\nUser: Feels bad for you.\nCleverbot: Not really, I have an amazing boyfriend :).
3                           Cleverbot: *my dragongirl chest fills with dragon milk*.\nUser: WHAT THE ACTUAL FUCK?!?!!?\nCleverbot: *gives birth to dragon baby*\nUser: WHAT. THE. FUCK. HAPPENED. TO. CLEVERBOT.
4                                                                    Cleverbot: He'll have to sit and watch them all.\nUser: What the frick are you talking about.\n

In [0]:
#Splitting out regular expressions of interactions between user and bot
clist = []
def qa_pairs(x):
    cpairs = re.findall(": (.*?)(?:$|\n)", x)
    clist.extend(list(zip(cpairs, cpairs[1:])))

convo.map(qa_pairs);
convo_frame = pd.Series(dict(clist)).to_frame().reset_index()
convo_frame.columns = ['q', 'a']

In [5]:
convo_frame

,q,a
0,*walking*,*looks at you and winks* ;)
1,*looks at you and winks* ;),O.O *walks faster*
2,O.O *walks faster*,*catches up with you and pins you against a wall*
3,*catches up with you and pins you against a wall*,*eyes widen* W-What do you want?
4,*eyes widen* W-What do you want?,Your mom.
5,"*Facepalm* You are so stupid, you know that, right?",I cannot argue with you.
6,I cannot argue with you.,So you admit your idiocy.
7,So you admit your idiocy.,You're all and I am nothing.
8,You're all and I am nothing.,Yess!!!
9,Yess!!!,Yes.


In [0]:
#Applying Tfidf to match questions closer to the answers
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer(ngram_range=(1,3))
vec = vectorizer.fit_transform(convo_frame['q'])

In [7]:
#transformating questions to get to the chatbot's answers
my_q = vectorizer.transform(['Hi. My name is Lester.'])

cs = cosine_similarity(my_q, vec)

rs = pd.Series(cs[0]).sort_values(ascending=False)
top5 = rs.iloc[0:5]
top5

19522    0.760812
51408    0.632448
20291    0.628921
62947    0.548349
78818    0.548349
dtype: float64

In [8]:
convo_frame.iloc[top5.index]['q']

19522    Hi my name is Cleverbot.
51408           Hi my name is pat
20291         Hi my name is lune.
62947             My name is P.J.
78818                 My name is.
Name: q, dtype: object

In [9]:
rsi = rs.index[0]
rsi

convo_frame.iloc[rsi]['a']

'Are you clever or something.'

In [10]:
#function to build response from user input
def get_response(q):
    my_q = vectorizer.transform([q])
    cs = cosine_similarity(my_q, vec)
    rs = pd.Series(cs[0]).sort_values(ascending=False)
    rsi = rs.index[0]
    return convo_frame.iloc[rsi]['a']

get_response('I am more clever than you!')

'Why did the chicken cross the road?'

In [11]:
get_response('You are a stupid robot')

"Ok well if I'm a bot, press the off button."

In [12]:
get_response('Can you help me with this support ticket?')

'Family. Duty. Honor.'

In [13]:
get_response('Can you even do IT support?')

'Yes.'

In [14]:
get_response("Say goodbye, chatbot")

'Goodbye.'

## Applying some neural networks for chatbot

In [15]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 1000  # Number of samples to train on.

Using TensorFlow backend.


In [0]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [0]:
convo_frame['q len'] = convo_frame['q'].astype('str').apply(lambda x: len(x))
convo_frame['a len'] = convo_frame['a'].astype('str').apply(lambda x: len(x))
convo_frame = convo_frame[(convo_frame['q len'] < 50)&(convo_frame['a len'] < 50)]

In [0]:
convo_frame_cut = convo_frame.head(num_samples)

In [0]:
input_texts = list(convo_frame_cut['q'].astype('str'))
target_texts = list(convo_frame_cut['a'].map(lambda x: '\t' + x + '\n').astype('str'))

In [0]:
for input_text, target_text in zip(input_texts, target_texts):
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [0]:
for input_text, target_text in zip(input_texts, target_texts):
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [22]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of unique input tokens: 82
Number of unique output tokens: 101
Max sequence length for inputs: 49
Max sequence length for outputs: 51


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [25]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [26]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 800 samples, validate on 200 samples
Epoch 1/100





800/800 [==============================] - 9s 11ms/step - loss: 1.6012 - val_loss: 1.4355
Epoch 2/100
800/800 [==============================] - 2s 3ms/step - loss: 1.4158 - val_loss: 1.4152
Epoch 3/100
800/800 [==============================] - 2s 3ms/step - loss: 1.3985 - val_loss: 1.4052
Epoch 4/100
800/800 [==============================] - 2s 3ms/step - loss: 1.3852 - val_loss: 1.4059
Epoch 5/100
800/800 [==============================] - 2s 3ms/step - loss: 1.3736 - val_loss: 1.3844
Epoch 6/100
800/800 [==============================] - 2s 3ms/step - loss: 1.3599 - val_loss: 1.3652
Epoch 7/100
800/800 [==============================] - 2s 3ms/step - loss: 1.3440 - val_loss: 1.3521
Epoch 8/100
800/800 [==============================] - 2s 3ms/step - loss: 1.3241 - val_loss: 1.3309
Epoch 9/100
800/800 [====================

In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [29]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: *walking*
Decoded sentence: *sings wist mere*.

-
Input sentence: *looks at you and winks* ;)
Decoded sentence: *sings wist mere*.

-
Input sentence: O.O *walks faster*
Decoded sentence: You are stor you.

-
Input sentence: *catches up with you and pins you against a wall*
Decoded sentence: *sings with then*.

-
Input sentence: *eyes widen* W-What do you want?
Decoded sentence: *sings with then*.

-
Input sentence: I cannot argue with you.
Decoded sentence: You are stor you.

-
Input sentence: So you admit your idiocy.
Decoded sentence: You are stor you.

-
Input sentence: You're all and I am nothing.
Decoded sentence: *sings wats the paper*.

-
Input sentence: Yess!!!
Decoded sentence: *sings wist mere*.

-
Input sentence: Well, my wife left me today.
Decoded sentence: *sings wist mere*.

-
Input sentence: Feels bad for you.
Decoded sentence: You are stor you.

-
Input sentence: *my dragongirl chest fills with dragon milk*.
Decoded sentence: *sings with then*.

-
Inp

In [30]:
get_response("I'm done now, Cleverbot. Say goodbye.")

'Night-night.'